In [9]:
# Generate Random CIK Numbers from SEC Website 

import pandas as pd
df = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
random_sample = df.sample(n=85)
company_list = random_sample.values.tolist()
cik_numbers = []
for i in company_list:
    cik_numbers.append(i[1])
print(cik_numbers)

[1817786, 1868640, 1811414, 1847846, 70502, 1059386, 1750593, 1370450, 1835817, 1289213, 1692705, 1812173, 1438893, 1863460, 1839730, 1067318, 1514743, 1627611, 1622231, 1530185, 1464423, 1822873, 1349929, 1357874, 1096056, 1376891, 1131343, 1591670, 1718108, 1674227, 1889123, 1780097, 1875444, 1332943, 1099590, 1297996, 1799011, 892832, 1858912, 1040829, 1506492, 354647, 1158324, 1842916, 1365794, 921671, 4281, 1482753, 1016281, 1510247, 1469443, 916540, 1855886, 314808, 1847888, 899611, 1161611, 1028918, 1600438, 277948, 1101239, 1635729, 1301236, 1690947, 1471824, 1817229, 1429937, 1493130, 1362516, 1593538, 1831915, 1817786, 1013880, 1411342, 1905448, 1398733, 1438893, 1818787, 1869824, 23795, 9631, 1845097, 912593, 1039828, 832489]


In [10]:
# Add 90 Days to 10-Q Filing Date

# import pandas, timedelta
import pandas as pd
from datetime import timedelta
   
# read csv data
df1 = pd.read_csv('10_Q_FILINGS.csv')
df1['date.filed'] = pd.to_datetime(df1['date.filed']) 
df1['90_DAYS'] = df1["date.filed"] + timedelta(days=90)
df2 = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
df2.columns = ['ticker', 'cik']
df2['ticker'] = df2['ticker'].str.upper()

   
df_merged = pd.merge(df1, 
                      df2, 
                      on ='cik', 
                      how ='inner')


quarterly_df = (df_merged[['cik','company.name', 'ticker', 'date.filed', '90_DAYS']])
#quarterly_df.to_csv('90_DAYS.csv')

[**********************99%***********************]  99 of 100 completed

In [11]:
# Get Stock Price for Quarterly Date + 90 Days

# Import yfinance and matplotlib
import yfinance as yf  
import matplotlib.pyplot as plt

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['date.filed'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': 'date.filed'})
df_filed_merged = pd.merge(quarterly_df, 
                      df_ohlc, 
                      how ='inner',
                      on =['date.filed', 'ticker'] )
df_filed_merged = df_filed_merged.rename(columns={'Close': 'Filed_Price'})
print(df_filed_merged)

[*********************100%***********************]  100 of 100 completed

14 Failed downloads:
- SOHON: CircuitBreaker 'redis' is OPEN and does not permit further calls
- SOHOO: CircuitBreaker 'redis' is OPEN and does not permit further calls
- IROQ: CircuitBreaker 'redis' is OPEN and does not permit further calls
- AEL: CircuitBreaker 'redis' is OPEN and does not permit further calls
- QS: CircuitBreaker 'redis' is OPEN and does not permit further calls
- BTIM: CircuitBreaker 'redis' is OPEN and does not permit further calls
- NSTC-WT: No data found for this date range, symbol may be delisted
- BXRX: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=BXRX, cluster=finance]]
- HWM-P: No data found for this date range, symbol may be delisted
- PV-WT: No data found for this date range, symbol may be delisted
- LAXXR: No data found for this date range, symbol may be delisted
- RBOT-WT: No data found for this date range, symbol may be delisted
- VAL-WT: No data fou

In [12]:
# Import yfinance and matplotlib
import yfinance as yf  
import matplotlib.pyplot as plt

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['90_DAYS'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': '90_DAYS'})
df_quarterly_merged = pd.merge(df_filed_merged, 
                      df_ohlc, 
                      how ='inner',
                      on =['90_DAYS', 'ticker'] )
df_quarterly_merged = df_quarterly_merged.rename(columns={'Close': '90_Days_Close_Price', 'Filed_Price': 'Filed_Close_Price'})
df_quarterly_merged['Price_Change'] = df_quarterly_merged['90_Days_Close_Price'] - df_quarterly_merged['Filed_Close_Price']
print(df_quarterly_merged)
#df_quarterly_merged.to_csv('stock_prices_90_days.csv')

[*********************100%***********************]  100 of 100 completed
         cik              company.name ticker date.filed    90_DAYS  \
0    1013880     TELETECH HOLDINGS INC   TTEC 2017-05-09 2017-08-07   
1    1013880     TELETECH HOLDINGS INC   TTEC 2017-11-08 2018-02-06   
2    1013880       TTEC Holdings  Inc    TTEC 2018-05-10 2018-08-08   
3    1013880       TTEC Holdings  Inc    TTEC 2018-08-07 2018-11-05   
4    1013880       TTEC Holdings  Inc    TTEC 2018-11-07 2019-02-05   
..       ...                       ...    ...        ...        ...   
592   916540  DARLING INGREDIENTS INC     DAR 2020-08-05 2020-11-03   
593   916540  DARLING INGREDIENTS INC     DAR 2020-11-03 2021-02-01   
594   916540  DARLING INGREDIENTS INC     DAR 2021-05-11 2021-08-09   
595   916540  DARLING INGREDIENTS INC     DAR 2021-08-10 2021-11-08   
596   916540  DARLING INGREDIENTS INC     DAR 2021-11-09 2022-02-07   

     Filed_Close_Price  90_Days_Close_Price  Price_Change  
0            3

In [15]:
#Calculate the Percent Change between the stock price at 10-Q filing date and after 90 days

# Import yfinance and matplotlib
import pandas as pd
import yfinance as yf  
import matplotlib.pyplot as plt
from datetime import timedelta

df1 = pd.read_csv('10_Q_FILINGS.csv')
df1['date.filed'] = pd.to_datetime(df1['date.filed']) 
df1['90_DAYS'] = df1["date.filed"] + timedelta(days=90)
df2 = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
df2.columns = ['ticker', 'cik']
df2['ticker'] = df2['ticker'].str.upper()

   
df_merged = pd.merge(df1, 
                      df2, 
                      on ='cik', 
                      how ='inner')


quarterly_df = (df_merged[['cik','company.name', 'ticker', 'date.filed', 'period.of.report', '90_DAYS']])

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['date.filed'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': 'date.filed'})
df_filed_merged = pd.merge(quarterly_df, 
                      df_ohlc, 
                      how ='inner',
                      on =['date.filed', 'ticker'] )
df_filed_merged = df_filed_merged.rename(columns={'Close': 'Filed_Price'})

df_ohlc = yf.download(df_merged['ticker'].unique().tolist(), start=df_merged['90_DAYS'].min())[['Close']]
df_ohlc = df_ohlc.stack(level=1).reset_index().rename(columns={'level_1':'ticker'})
df_ohlc = df_ohlc.rename(columns = {'Date': '90_DAYS'})
df_quarterly_merged = pd.merge(df_filed_merged, 
                      df_ohlc, 
                      how ='inner',
                      on =['90_DAYS', 'ticker'] )
df_quarterly_merged = df_quarterly_merged.rename(columns={'Close': '90_Days_Close_Price', 'Filed_Price': 'Filed_Close_Price'})
df_quarterly_merged['Price_Change'] = df_quarterly_merged['90_Days_Close_Price'] - df_quarterly_merged['Filed_Close_Price']
def percentage_change(col1,col2):
    return ((col2 - col1) / col1)
df_quarterly_merged['Pct_Change_Price'] = percentage_change(df_quarterly_merged['Filed_Close_Price'],df_quarterly_merged['90_Days_Close_Price'])
print(df_quarterly_merged)
#df_quarterly_merged.to_csv('stock_prices_90_days_pct_changev2.csv')

[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed

10 Failed downloads:
- HWM-P: No data found for this date range, symbol may be delisted
- PV-WT: No data found for this date range, symbol may be delisted
- CSX: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=CSX, cluster=finance]]
- LAXXR: No data found for this date range, symbol may be delisted
- PMT-PB: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=PMT-PB, cluster=finance]]
- LUCD: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=LUCD, cluster=finance]]
- VAL-WT: No data found for this date range, symbol may be delisted
- RCHGU: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=RCHGU, cluster=finance]]
- RBOT-WT: No data found for this date range, symbol may be delisted
- PPHPR: No data found for this date range, symbol may be 

In [ ]:
#Get Free Cash Flow for Sample Companies from MacroTrends financial charts website

"""
Created on Tue Apr  5 20:36:06 2022

@author: conno
"""
from enum import unique
from numpy import inner
import requests
import time
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta


def getAllFCF(df):
    tickers = df['ticker'].unique()
    allfcf = pd.DataFrame()
    for i in tickers:
        url = df.loc[df['ticker'] == i, 'url']
        fcf = getFCF(i, url.iloc[0])
        fcf['Ticker'] = i
        allfcf = allfcf.append(fcf, ignore_index=True)
    return allfcf


def getFCF(ticker, url):
    page = requests.get("https://www.macrotrends.net/stocks/charts/" + ticker + "/" + url + "/free-cash-flow")
    if page.status_code != 200:
        return pd.DataFrame()
    else:    
    
        soup = BeautifulSoup(page.content, 'html.parser')
        tables = soup.find_all(class_="historical_data_table")
        text = tables[1]
        print(text.prettify())

        table_rows = text.find_all('tr')

        rows = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [i.text for i in td]
            rows.append(row)

        df = pd.DataFrame(rows, columns =['Date', 'Value'])
        return df

def convertCompanyNames(names, df):
    lst = []
    for i in names:
        name = i.lower()
        name = name.replace(" ","-")
        lst.append([i,name])
        
    df2 = pd.DataFrame(lst, columns=["company.name", "url"])
    return pd.merge(df,df2,on="company.name", how="inner")


def main():
    df1 = pd.read_csv('10_Q_FILINGS.csv')
    df1['date.filed'] = pd.to_datetime(df1['date.filed']) 
    df1['90_DAYS'] = df1["date.filed"] + timedelta(days=90)
    df2 = pd.read_csv('https://www.sec.gov/include/ticker.txt', sep="\t", header=None)
    df2.columns = ['ticker', 'cik']
    df2['ticker'] = df2['ticker'].str.upper()

    
    df_merged = pd.merge(df1, 
                        df2, 
                        on ='cik', 
                        how ='inner')


    #print(df_merged[['company.name', 'date.filed', 'ticker', '90_DAYS']])
    companies = df_merged['company.name'].unique()

    df3 = convertCompanyNames(companies, df_merged)

    
    df3['url'] = df3['url'].str.replace('-$', '')
    df3['url'] = df3['url'].str.replace('--', '-')
    
    allfcf = getAllFCF(df3)
    #file_output = allfcf.to_csv('output_fcf.csv')


if __name__ == "__main__":
    main()

In [ ]:
# Run main function
main()

In [16]:
#Calculate Free Cash Flow Percent Change from Previous Quarter

import pandas as pd
df_pct_change = pd.read_csv('output_fcf.csv', index_col=0)
df_pct_change = df_pct_change.dropna(how='any',axis=0) 
df_pct_change['Value'] = pd.to_numeric(df_pct_change['Value'],errors='coerce')
df_pct_change['pct'] = df_pct_change.sort_values('Date', axis=0, ascending=True).groupby(['Ticker']).Value.pct_change()
#df_pct_change.to_csv('fcf_pct_change.csv')
print(df_pct_change)

            Date   Value Ticker        pct
1     2021-09-30  134.33   TTEC   0.229228
2     2021-06-30  109.28   TTEC   0.876052
3     2021-03-31   58.25   TTEC  -0.725456
4     2020-12-31  212.17   TTEC   0.526732
5     2020-09-30  138.97   TTEC   0.893840
...          ...     ...    ...        ...
1584  2010-03-31   10.22    DAR  -0.822137
1585  2009-12-31   57.46    DAR   0.544209
1586  2009-09-30   37.21    DAR   0.679910
1587  2009-06-30   22.15    DAR  16.038462
1588  2009-03-31    1.30    DAR        NaN

[1493 rows x 4 columns]


In [14]:
#Merge Percent Change of Free Cash Flow with Stock Price Change Dataframe


import pandas as pd
df_stock_prices = pd.read_csv('stock_prices_90_days_pct_changev2.csv')
df_fcf = pd.read_csv('fcf_pct_change.csv', index_col=0)
df_stock_prices = df_stock_prices.rename(columns = {'period.of.report': 'Date', 'ticker': 'Ticker'})
df_fcf_merged = pd.merge(df_fcf, 
                      df_stock_prices, 
                      how ='inner',
                      on =['Date','Ticker'] )

df_fcf_merged.drop(['Unnamed: 0'], axis=1)
df_fcf_merged = df_fcf_merged.rename(columns = {'Date': 'Period', 'pct': 'FCF_PCT_CHANGE', 'Value': 'FCF'})
print(df_fcf_merged)
df_fcf_merged.to_csv('stock_prices_90_days_FCF.csv')

         Period     FCF Ticker  FCF_PCT_CHANGE  Unnamed: 0      cik  \
0    2021-09-30  134.33   TTEC        0.229228          12  1013880   
1    2021-06-30  109.28   TTEC        0.876052          11  1013880   
2    2021-03-31   58.25   TTEC       -0.725456          10  1013880   
3    2020-09-30  138.97   TTEC        0.893840           9  1013880   
4    2020-06-30   73.38   TTEC        0.617368           8  1013880   
..          ...     ...    ...             ...         ...      ...   
402  2017-06-30  -10.94    SUI        3.695279         616   912593   
403  2017-03-31   -2.33    SUI       -1.324061         615   912593   
404  2018-06-30   -8.84    DAR       -0.686302         634   916540   
405  2018-03-31  -28.18    DAR       -1.195206         633   916540   
406  2017-09-30   77.28    DAR        0.054010         632   916540   

                 company.name  date.filed     90_DAYS  Filed_Close_Price  \
0         TTEC Holdings  Inc   2021-11-09  2022-02-07         100.94000

In [5]:
import yfinance as yf
import numpy as np
from pandas_datareader import data
import pandas as pd
import pandas_datareader as web
import numpy as np

df=pd.read_csv('stock_prices_90_days_FCF.csv')
df_tickers = df['Ticker'].drop_duplicates()
tickers = df_tickers.values.tolist()
market_data=[]
for ticker in tickers:
           try: 
               market_data.append(web.get_quote_yahoo(ticker)['marketCap'])
           except:
               print('Error with: ', ticker)
df=pd.concat(market_data, axis=0)
#df = df.rename_axis('Ticker')
df = pd.DataFrame({'Ticker':df.index, 'Market_Cap':df.values})
df['Market_Cap_Size'] = pd.cut(df['Market_Cap'], [0, 2000000000, 10000000000, np.inf], labels=['small', 'medium', 'large'])
df.to_csv('marketcap.csv', index=False)
print(df)


   Ticker   Market_Cap Market_Cap_Size
0    TTEC   3668776704          medium
1     CSV    802908992           small
2    PPBI   3387150848          medium
3     AEL   3893479936          medium
4     RHP   5323194880          medium
5    LXFR    469383072           small
6    MELI  51125370880           large
7    EQIX  68954783744           large
8    CCOI   3141525760          medium
9     DLR  43658883072           large
10   SOHO     71824336           small
11   VYNT     32647764           small
12   DTIL    140388080           small
13   WLDN    375018272           small
14   AQST     84980696           small
15    EFC    974450432           small
16    PMT   1420827520           small
17   RKDA     23742124           small
18   NUWE      9698807           small
19   IROQ     77090560           small
20    FPI    736086080           small
21   NAVI   2554312960          medium
22    GMS   2187343616          medium
23   RETC      2650180           small
24   WORX     11497335   

In [16]:
import yfinance as yf
import numpy as np
from pandas_datareader import data
import pandas as pd
import pandas_datareader as web
import numpy as np
from yahooquery import Ticker

df = pd.read_csv('marketcap.csv')
df_tickers = df['Ticker'].drop_duplicates()
tickers = df_tickers.values.tolist()
sectors = []
for ticker in tickers:
  sectors.append(Ticker(ticker).summary_profile[ticker]['sector'])
df['sector'] = sectors
print(df)
df.to_csv('market_cap_sector.csv')



   Ticker   Market_Cap Market_Cap_Size                  sector
0    TTEC   3668776704          medium              Technology
1     CSV    802908992           small       Consumer Cyclical
2    PPBI   3387150848          medium      Financial Services
3     AEL   3893479936          medium      Financial Services
4     RHP   5323194880          medium             Real Estate
5    LXFR    469383072           small             Industrials
6    MELI  51125370880           large       Consumer Cyclical
7    EQIX  68954783744           large             Real Estate
8    CCOI   3141525760          medium  Communication Services
9     DLR  43658883072           large             Real Estate
10   SOHO     71824336           small             Real Estate
11   VYNT     32647764           small              Healthcare
12   DTIL    140388080           small              Healthcare
13   WLDN    375018272           small             Industrials
14   AQST     84980696           small              Hea

In [1]:
import pandas as pd

df1 = pd.read_csv('stock_prices_90_days_FCF.csv')
df2 = pd.read_csv('market_cap_sector.csv')

df_sector_merged = pd.merge(df1, 
                      df2, 
                      how ='inner',
                      on =['Ticker'] )
def fcf_change(row):
  if row['FCF_PCT_CHANGE'] < 0:
    return 'Decrease'
  else:
    return 'Increase'
def price_change(row):
  if row['Pct_Change_Price'] < 0:
    return 'Decrease'
  else:
    return 'Increase'
df_sector_merged['FCF_CHANGE'] = df_sector_merged.apply(fcf_change, axis=1)
df_sector_merged['PRICE_CHANGE'] = df_sector_merged.apply(price_change, axis=1)
df_sector_merged.to_csv('sector_merged.csv')
print(df_sector_merged)


     Unnamed: 0_x      Period     FCF Ticker  FCF_PCT_CHANGE  Unnamed: 0.1  \
0               0  2021-09-30  134.33   TTEC        0.229228            12   
1               1  2021-06-30  109.28   TTEC        0.876052            11   
2               2  2021-03-31   58.25   TTEC       -0.725456            10   
3               3  2020-09-30  138.97   TTEC        0.893840             9   
4               4  2020-06-30   73.38   TTEC        0.617368             8   
..            ...         ...     ...    ...             ...           ...   
402           402  2017-06-30  -10.94    SUI        3.695279           616   
403           403  2017-03-31   -2.33    SUI       -1.324061           615   
404           404  2018-06-30   -8.84    DAR       -0.686302           634   
405           405  2018-03-31  -28.18    DAR       -1.195206           633   
406           406  2017-09-30   77.28    DAR        0.054010           632   

         cik              company.name  date.filed     90_DAYS 